In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
# Define the required columns for feature selection
req_cols = [' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes', ' Destination Port', 
            ' Packet Length Variance', ' Bwd Packet Length Mean', ' Avg Bwd Segment Size', 'Bwd Packet Length Max', 
            ' Init_Win_bytes_backward', 'Total Length of Fwd Packets', ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', 
            ' Average Packet Size', ' Packet Length Mean', ' Max Packet Length',' Label']

# Load the data from csv files
fraction = 1
frames = []

for filename in ['Wednesday-workingHours.pcap_ISCX.csv', 'Tuesday-WorkingHours.pcap_ISCX.csv', 
                 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
                 'Monday-WorkingHours.pcap_ISCX.csv', 'Friday-WorkingHours-Morning.pcap_ISCX.csv', 
                 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv']:
    
    df = pd.read_csv(f'cicids_db/{filename}', usecols=req_cols).sample(frac=fraction)
    frames.append(df)

df = pd.concat(frames, ignore_index=True)



df_max_scaled = df.copy()
y = df_max_scaled[' Label'].replace({'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos', 'DDoS': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})
df_max_scaled.pop(' Label')
df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

df['Init_Win_bytes_forward'] = df['Init_Win_bytes_forward'] + 1
df[' Init_Win_bytes_backward'] = df[' Init_Win_bytes_backward'] + 1

df['Init_Win_bytes_forward'] = df['Init_Win_bytes_forward']/df['Init_Win_bytes_forward'].max()
df[' Init_Win_bytes_backward'] = df[' Init_Win_bytes_backward']/df[' Init_Win_bytes_backward'].max()







In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
# Define the features and target variable
X = df.drop('Label', axis=1)
y = df['Label']

# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=10, random_state=42)
clf.fit(X, y)

# Get feature importances and sort them in descending order
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Select top 5 features based on importance score
top_features = X.columns[indices][:10]

print(top_features)

